In [ ]:
import numpy as np
import pandas as pd
import torch
import itertools
from sklearn.metrics import adjusted_mutual_info_score
from scipy.stats import special_ortho_group

import tools

In [ ]:
d = 30
df = pd.read_csv('data/large.csv')
n_samples = len(df)
expected_features = np.array([3, 8])

In [ ]:
features = [f'f{n}' for n in range(d)]
targets = ['y']
xdf = df[features]
ydf = df[targets]
x = xdf.values
y = ydf.values

## Uncover the dependence between target and features

We check that 
$$
y =
\begin{cases}
1 & \text{  if } x_{k_{0}} = x_{k_{1}}
\\
0 & \text { otherwise},
\end{cases}
$$
where $k_0 = 3$ and $k_1 = 8$
are the expected features.

From the thirty features $0, \dots, 29$ in `df`, our feature selection is exact if it keeps features 3 and 8, and it discards all others.

In [ ]:
test = np.array(x[:, expected_features[0]] == x[:, expected_features[1]], dtype=int)
assert np.all(test == y[:, 0])

## Preliminary check: expected features bear the highest information content

Of all $d \choose 2$ pairs of features, we check that the expected pair $\lbrace 3 , 8 \rbrace$ has the highest mutual information with the target.

In [ ]:
l = 2
miscores = {subset: 
           adjusted_mutual_info_score(tools.onedimlabel(x[:, list(subset)]), y[:, 0])
            for subset in itertools.combinations(list(range(d)), l)
            
}

In [ ]:
s = (0,1)
mi = 0
for k, v in miscores.items():
    if v > mi:
        s = k
        mi = v
highest_info = s

In [ ]:
print(f'Expected features: {sorted(expected_features)}')
print(f'Pair of features with highest information content: {sorted(highest_info)}')

### Selection with marginal 1D ksg mutual info

In [ ]:
ksgselection, mis = tools.ksgmi(xdf, ydf, threshold=0.05)

In [ ]:
print(f'Expected features: {sorted(expected_features)}')
print(f'Marginal KSG selection: {sorted(ksgselection)}')

### Selection with HSIC Lasso

In [ ]:
xfeattype = tools.FeatureType.CATEGORICAL
yfeattype = tools.FeatureType.CATEGORICAL
hsiclasso_selection = tools.pyhsiclasso(
    x, y, xfeattype=xfeattype, yfeattype=yfeattype, n_features=2, batch_size=500)

In [ ]:
print(f'Expected features: {sorted(expected_features)}')
print(f'HSIC Lasso selection: {sorted(hsiclasso_selection)}')

### Selection with Boruta

In [ ]:
from arfs.feature_selection import allrelevant
from arfs.feature_selection.allrelevant import Leshy
from sklearn.ensemble import RandomForestClassifier

In [ ]:
n_estimators = 'auto'
perc = 95
alpha = 0.05
importance = "shap"
two_step = True
max_iter = 100
random_state = 1234
verbose = 0
keep_weak = False

In [ ]:
xdf = pd.DataFrame(x, columns = [f'f{i}' for i in range(d)])
yser = pd.Series(y[:, 0], name='y')

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, max_depth=8)

In [ ]:
leshy = Leshy(
    rf,
    n_estimators=n_estimators,
    perc=perc,
    alpha=alpha,
    importance=importance,
    two_step=two_step,
    max_iter=max_iter,
    random_state=random_state,
    verbose=verbose,
    keep_weak=keep_weak,
)

In [ ]:
leshy.fit(xdf, yser)
leshy_selection = [int(col.replace('f', '')) for col in leshy.selected_features_]

In [ ]:
print(f'Expected features: {sorted(expected_features)}')
print(f'Boruta selection: {sorted(leshy_selection)}')

## Selection with Minerva

In [ ]:
import minerva

In [ ]:
feature_cols = [f'f{n}' for n in range(d)]
cat_features = feature_cols  # all features are categorical
float_features = []  # no feature is float
targets = ['y']
cat_feat_sizes = 1 + df[cat_features].max().values
train_size = int(.75 * n_samples)
val_size = int(.225 * n_samples)
test_size = n_samples - train_size - val_size

In [ ]:
train_data = df.iloc[:train_size]
val_data = df.iloc[train_size: train_size + val_size]
test_data = df.iloc[:-test_size]

In [ ]:
dimension_of_residual_block = 512 
num_res_layers = 4 
scaler = 2  
batch_size = scaler*1200
num_batches = n_samples // batch_size
max_epochs = int(2000*scaler)  
lr = 5e-6  
emb_dim = 4 
reg_coef = 1e6 

In [ ]:
# Pack hyperparameters
selector_params = dict(
    cat_features=cat_features,
    float_features=float_features,
    targets=targets,
    dim1_max=dimension_of_residual_block,
    lr=lr,
    num_res_layers=num_res_layers,
    eps=.001,
    cat_feat_sizes=cat_feat_sizes,
    emb_dim=emb_dim,
)
logger_params = dict(
    name="experiment_1"
)

In [ ]:
# Set dataloaders
train_dataloader, val_dataloader, test_dataloader = minerva.feature_selection.dataloaders(
    train_data=train_data,
    val_data=val_data,
    test_data=test_data,
    float_features=float_features,
    categorical_features=cat_features,
    targets=targets,
    batch_size=batch_size,
) 

In [ ]:
logs = []

In [ ]:
# First pass: No regularisation
noreg_path = 'data/noreg.model'
out, selector = minerva.feature_selection.train(
    selector_params=selector_params,
    logger_params=logger_params,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    reg_coef=.0,
    projection_init=.20,
    disable_projection=False,
    max_epochs=max_epochs,
    load_path=None
)   
logs.append(out)
torch.save(selector.state_dict(), noreg_path)

In [ ]:
previous_segment_path = noreg_path
# Second pass: Apply regularisation
for segment in range(5):
    out, selector = minerva.feature_selection.train(
        selector_params=selector_params,
        logger_params=logger_params,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        test_dataloader=test_dataloader,
        reg_coef=reg_coef,
        disable_projection=False,
        max_epochs=max_epochs,
        load_path=previous_segment_path
    )
    segment_path = f'data/trained.model.{segment}.0'
    torch.save(selector.state_dict(), segment_path)
    logs.append(out)
    previous_segment_path = segment_path

dflogs = pd.DataFrame(logs)
dflogs.to_csv('data/traininglogs.csv', index=False)